In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

%matplotlib inline

# Hodgkin-Huxley

Neuron dynamic can be described by three dimensional model 

\begin{equation}
C\frac{dV}{dt}=I_{\text{L}}+I_{\text{K}}+I_{\text{N}}+I_{\text{S}}
\end{equation}

The leak-current $I_{\text{L}}$ is given by 

\begin{equation}
I_{\text{L}}=-g_{L}(V-V_{\text{R}})
\end{equation}

where $g_{L}$ is conductance of the leak-channel and $V_{\text{R}}$ is the resting pontential. The conductance $g_{L}$ is approximated as constant. To better understand the role of the leak-channel, we neglect the other currents and such that

\begin{equation}
C\frac{dV}{dt} = -g_{L}(V-V_{\text{R}})
\end{equation}

The above equation tell us that indepdently of its initial value, the membrane potential decays exponentially towards the resting potential $V_r$. This is shown in the figure below for two different initial values. 

In [ ]:
C   = 250 # capacitance in PF
tau = 10  # membrane time constant in ms
V_r = -65 # resting potential in mV

fz = 18

t = np.linspace(0, 5*tau, int(1e4))

V_0 = -100 # initial membrane potential in mV 
V = V_0*np.exp(-t/tau) + (1 - np.exp(-t/tau))*V_r

inch = 2.54
plt.figure(figsize = (12., 8.))

plt.plot(t, V, '-', color = 'b')
                          
V_0 = -30 # initial membrane potential in mV 
V = V_0*np.exp(-t/tau) + (1 - np.exp(-t/tau))*V_r
                          
plt.plot(t, V, '-', color = 'r')
                          
plt.plot([t[0], t[-1]], [V_r, V_r], '--', color = 'k')
plt.xlabel('time in ms', fontsize = fz)
plt.ylabel('membrane potential in mV', fontsize = fz)

plt.xlim([t[0], t[-1]])

plt.show()
                        

Next we will look at the Potassium channel 

\begin{equation}
I_{\text{K}}=g_{\text{K}}(V_{\text{K}}-V)
\end{equation}

The conductance $g_{\text{K}}$ depends on the membrane potential   

\begin{align}
g_{\text{K}}&=\bar{g}_{\text{K}}m^{3}h \\
g_{\text{N}}&=\bar{g}_{\text{N}}n^{4}
\end{align}

through the gating variables $h$ and $m$. The variables are described by first oder kintetics 

\begin{align}
\frac{dm}{dt}&=-\frac{1}{\tau_{m}(V)}(m - m_{\infty}(V)), \\
\frac{dh}{dt}&=-\frac{1}{\tau_{h}(V)}(h - h_{\infty}(V)).
\end{align}

For a fixed value of $V$, $m(t)$ decays exponentially to $m_{\infty}(V)$ with a time constant $\tau_{m}(V)$. The variables $m$ and $n$ are called an activation variable. This terminology can be understood from the fact that the value $m_{\infty}(V)$ is close to zero. Therefore, at rest, the sodium and potassium current $I_{\text{Na}}$ through the channel vanishes. The variable $h$ is deactivation variable, it operators on a longer time scale and closes the sodium channel after spike initiation.    

In [ ]:
V_Na = 55 # reversal potential potassium in mV
V_K  = -77 # reversal pential sodium in mV

g_K  = 40 # in mS
g_Na = 35 # in mS


def alpha_n(V):
    
    return 0.02*(V-25.)/(1.- np.exp(-(V-25.)/9.))
        
def beta_n(V):
    
    return -0.002*(V-25.)/(1. - np.exp((V-25.)/9.))
    
def alpha_m(V):

    return 0.182*(V+35.)/(1.- np.exp(-(V+35.)/9.)) 
            
def beta_m(V):
    
    return -0.124*(V+35.)/(1. - np.exp((V+35.)/9.)) 

def alpha_h(V):
   
    return 0.25*np.exp(-(V+90.)/12.)
                           
def beta_h(V):
                           
    return 0.25*np.exp((V + 62.)/6.)/np.exp((V+90.)/12.)
                           
def n_inf(V):

    return alpha_n(V)/(alpha_n(V) + beta_n(V))

def m_inf(V):
        
    return alpha_m(V)/(alpha_m(V) + beta_m(V))

def h_inf(V):

    return alpha_h(V)/(alpha_h(V) + beta_h(V))

def tau_m(V):
    
    return 1./(alpha_m(V) + beta_m(V))

def tau_n(V):
    
    return 1./(alpha_n(V) + beta_n(V))

def tau_h(V):
    
    return 1./(alpha_h(V) + beta_h(V))

    
V_arr = np.linspace(-100, 100, 1000)

n_inf_arr = n_inf(V_arr)
m_inf_arr = m_inf(V_arr)
h_inf_arr = h_inf(V_arr)

tau_n_arr = tau_n(V_arr)
tau_m_arr = tau_m(V_arr)
tau_h_arr = tau_h(V_arr)

plt.figure(figsize = (12., 8.))
gs  = plt.GridSpec(2,1)

ax0 = plt.subplot(gs[0])
ax0.plot([V_r, V_r], [0,1.05], '--', c = 'k', lw = 1.0)

ax0.plot(V_arr, n_inf_arr, c = 'y', label = 'n')
ax0.plot(V_arr, m_inf_arr, c = 'g', label = 'm')
ax0.plot(V_arr, h_inf_arr, c = 'c', label = 'h')

ax0.set_ylabel(u'$x_{\infty}(V)$', fontsize = fz)

lfz = 16

ax0.set_ylim([0,1.05])
ax0.set_xlim([V_arr[0],V_arr[-1]])
ax0.legend(fontsize = lfz)

ax1 = plt.subplot(gs[1])

tau_max = 1.05*np.max(tau_h_arr)

ax1.plot([V_r, V_r], [0,tau_max], '--', c = 'k', lw = 1.0)
ax1.plot(V_arr, tau_n_arr, c = 'y')
ax1.plot(V_arr, tau_m_arr, c = 'g')
ax1.plot(V_arr, tau_h_arr, c = 'c')

ax1.set_ylim([0, tau_max])
ax1.set_xlim([V_arr[0],V_arr[-1]])
ax1.set_ylabel(u'$\\tau_{x}(V)$', fontsize = fz)
ax1.set_xlabel('membrane potential in mV', fontsize = fz)

plt.show()

Solving the equation yields 

In [ ]:
# Set random seed (for reproducibility)
np.random.seed(1000)

# Start and end time (in milliseconds)
tmin = 0.0
tmax = 50.0

# # Nest parameters
# Cm  = 1. # membrane capacitance (pF)
# VK  = -77. # potassium potential (mV)
# VNa = 55. # sodium potential (mV)
# VL  = -65. # resting potential (mV)

# gL  = 0.3 # conductance leak-channel (nS)
# gK  = 50. # maximum conductance potassium channel (nS)
# gNa = 40. # maximum conductance sodium channel (nS) 
  
# Potassium ion-channel rate functions
def alpha_n(Vm):
    return (0.01 * (10.0 - Vm)) / (np.exp(1.0 - (0.1 * Vm)) - 1.0)

def beta_n(Vm):
    return 0.125 * np.exp(-Vm / 80.0)

# Sodium ion-channel rate functions
def alpha_m(Vm):
    return (0.1 * (25.0 - Vm)) / (np.exp(2.5 - (0.1 * Vm)) - 1.0)

def beta_m(Vm):
    return 4.0 * np.exp(-Vm / 18.0)

def alpha_h(Vm):
    return 0.07 * np.exp(-Vm / 20.0)

def beta_h(Vm):
    return 1.0 / (np.exp(3.0 - (0.1 * Vm)) + 1.0)
  
# n, m, and h steady-state values

def n_inf(Vm=0.0):
    return alpha_n(Vm) / (alpha_n(Vm) + beta_n(Vm))

def m_inf(Vm=0.0):
    return alpha_m(Vm) / (alpha_m(Vm) + beta_m(Vm))

def h_inf(Vm=0.0):
    return alpha_h(Vm) / (alpha_h(Vm) + beta_h(Vm))    
    
# Average potassium channel conductance per unit area (mS/cm^2)
gK = 36.0

# Average sodoum channel conductance per unit area (mS/cm^2)
gNa = 120.0

# Average leak channel conductance per unit area (mS/cm^2)
gL = 0.3

# Membrane capacitance per unit area (uF/cm^2)
Cm = 1.0

# Potassium potential (mV)
VK = -12.0

# Sodium potential (mV)
VNa = 115.0

# Leak potential (mV)
VL = 10.613  

# Compute derivatives
def compute_derivatives(y, t, I0):
    
    # Input current (pA)
    def Id(t):
        if 1.0 < t < 2.0:
            return I0
        return 0.0
        
    dy = np.zeros((4,))
    
    Vm = y[0]
    n = y[1]
    m = y[2]
    h = y[3]
    
    # dVm/dt
    GK  = gK/Cm * np.power(n, 4.0)
    GNa = gNa/Cm * np.power(m, 3.0) * h
    GL  = gL/Cm
    
    dy[0] = Id(t)/Cm  - GK*(Vm-VK) - GNa*(Vm-VNa) - GL*(Vm-VL)
    
    # dn/dt
    dy[1] = alpha_n(Vm) * (1.0 - n) - (beta_n(Vm) * n)
    
    # dm/dt
    dy[2] = alpha_m(Vm) * (1.0 - m) - (beta_m(Vm) * m)
    
    # dh/dt
    dy[3] = alpha_h(Vm) * (1.0 - h) - beta_h(Vm) * h
    
    return dy
  
# State (Vm, n, m, h)
Y = np.array([VL, n_inf(VL), m_inf(VL), h_inf(VL)])

# amplitudes of input current 
I_arr = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

# caculate response 
y_arr = []

for I in I_arr:
    
    y_arr.append(odeint(compute_derivatives, Y, T, args = (I,)))

# Input current (pA)
def Id(t, I):
    if 10.0 < t < 11:
        return I
    return 0.0

plt.figure(figsize = (14, 8))
gs = plt.GridSpec(3,1)    
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
ax2 = plt.subplot(gs[2])

for i, I in enumerate(I_arr):

    # Input stimulus
    Idv = [Id(t, I) for t in T]
            
    ax0.plot(T, Idv)
    
    v_arr = y_arr[i]
    
    ax1.plot(T, v_arr[:, 0])
    ax2.plot(T, v_arr[:, 0])
    
ax0.set_xlabel('Time (ms)')
ax0.set_ylabel(r'Current density (uA/$cm^2$)')
ax0.grid(True)
        
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Vm (mV)')
plt.grid(True)    

ax2.set_xlim([10, 13])
ax2.set_ylim([-65, -50])
